In [17]:
%load_ext autoreload
%autoreload 2

Load strain design data

In [18]:
import sys

sys.path.append("../src")

import matplotlib.pyplot as plt
import pandas as pd

from src.dataloader import DataLoader
from src.machinelearning import MachineLearning
from src.optimization import Optimization

# Load the data
DBTL_A = [
    "2X.Mh",
    "B.Lm",
    "2X.Ll",
    "A.Mm",
    "B.Ll",
    "A.Mh",
    "2X.Lm",
    "A.Hl",
    "2X.Hh",
    "B.Ml",
    "B.Mm",
    "2X.Lh",
    "B.Mh",
    "2X.Hl",
    "B.Hl",
    "2X.Ml",
    "B.Hm",
    "B.Lh",
    "B.Hh",
    "A.Ll",
    "A.Hm",
    "2X.Mm",
    "A.Hh",
    "A.Ml",
    "A.Lm",
    "A.Lh",
    "2X.Hm",
]
DBTL_B = ["BL.Mm", "BL.Mh", "BL.Ml"]
FILENAME = "preprocessed_Limonene_data.csv"

data = DataLoader(FILENAME, target="Limonene")
df = data.df.copy()
df_A = df.loc[DBTL_A]
df_B = df.loc[DBTL_B]

print(f"DBTL_A shape: {df_A.shape}")
print(f"DBTL_B shape: {df_B.shape}")

ML-based optimization

In [ ]:
x, y, x_offset, x_factor, y_offset, y_factor, scaled_lb, scaled_ub = data.normalization(
    df=df_A
)

In [ ]:
EPOCHS = 100
BATCH_SIZE = 4
VALIDATION_SPLIT = 0.33

net = MachineLearning(
    input_dim=x.shape[1],
    output_dim=y.shape[1],
    num_hidden=3,
    hidden_dim=3,
    activation="sigmoid",
    lr=0.01,
)

net.create_model()
net.train(x, y, EPOCHS, BATCH_SIZE, VALIDATION_SPLIT)
net.loss_curve()
net.save_model("reformer_nn.keras")

In [ ]:
# # Add 2 to all x_offset keys values
# x_offset = {key: value + 2 for key, value in x_offset.items()}
# y_offset = {key: value + 10 for key, value in y_offset.items()}

# x_factor = {key: value * 2 for key, value in x_factor.items()}
# y_factor = {key: value * 2 for key, value in y_factor.items()}

# scaled_lb = scaled_lb * .005
# scaled_ub = scaled_ub * 1.5

In [ ]:
omlt = Optimization(
    model_file="reformer_nn.keras", inputs=data.inputs, outputs=data.outputs
)
omlt.set_scaler(x_offset, x_factor, y_offset, y_factor, scaled_lb, scaled_ub)
omlt.load_net()
omlt.solve()

__Gradient Boosting Trees__

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pyomo.environ as pe
from omlt import OffsetScaling, OmltBlock
from omlt.gbt import GBTBigMFormulation, GradientBoostedTreeModel
from omlt.gbt.model import GradientBoostedTreeModel
from omlt.io.keras import load_keras_sequential
from omlt.neuralnet import FullSpaceSmoothNNFormulation
from onnxmltools.convert import convert_sklearn, convert_xgboost
from skl2onnx import to_onnx
from skl2onnx.common.data_types import FloatTensorType
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

'_BlockData'. The class '_BlockData' has been renamed to 'BlockData'.
(deprecated in 6.7.2) (called from
c:\Users\mexis\anaconda3\envs\main\lib\site-packages\omlt\block.py:33)



In [2]:
# Load the data
DBTL_A = ["2X.Mh", "B.Lm", "2X.Ll", "A.Mm", "B.Ll", "A.Mh", "2X.Lm", "A.Hl", "2X.Hh", "B.Ml", "B.Mm", "2X.Lh", "B.Mh", "2X.Hl", "B.Hl", "2X.Ml", "B.Hm", "B.Lh", "B.Hh", "A.Ll", "A.Hm", "2X.Mm", "A.Hh", "A.Ml", "A.Lm", "A.Lh", "2X.Hm"]
DBTL_B = ["BL.Mm", "BL.Mh", "BL.Ml"]
FILENAME = "preprocessed_Limonene_data.csv"

df = pd.read_csv(FILENAME, index_col=0)

inputs = df.drop(columns=["Limonene"]).columns.values.tolist()
outputs = ["Limonene"]

df_A = df.loc[DBTL_A]
df_B = df.loc[DBTL_B]

print(f"DBTL_A shape: {df_A.shape}")
print(f"DBTL_B shape: {df_B.shape}")

DBTL_A shape: (27, 10)
DBTL_B shape: (3, 10)


In [3]:
# Data scaling and scaler creation
dfin = df_A[inputs]
dfout = df_A[outputs]

x_offset, x_factor = dfin.mean().to_dict(), dfin.std().to_dict()
y_offset, y_factor = dfout.mean().to_dict(), dfout.std().to_dict()

dfin = (dfin - dfin.mean()).divide(dfin.std())
dfout = (dfout - dfout.mean()).divide(dfout.std())

scaled_lb = dfin.min()[inputs].values
scaled_ub = dfin.max()[inputs].values

scaler = OffsetScaling(
        offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
        factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
        offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
        factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))}
    )

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

In [4]:
def tune_and_train_rf(X_train, y_train):
    param_grid = {
        "n_estimators": np.linspace(3, 100, 100, dtype=int),
        "max_depth": np.linspace(2, 25, 100, dtype=int),
        "min_samples_split": np.linspace(0.1, 1.0, 100),
        "min_samples_leaf": np.linspace(0.1, 0.5, 100)
    }
    
    grid = RandomizedSearchCV(RandomForestRegressor(), param_distributions=param_grid,
                                n_iter=500, cv=3, verbose=3, n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid.best_estimator_

def tune_and_train_gbt(X_train, y_train):
    param_grid = {
        "n_estimators": np.linspace(3, 100, 100, dtype=int),
        "max_depth": np.linspace(2, 25, 100, dtype=int),
        "min_samples_split": np.linspace(0.1, 1.0, 100),
        "min_samples_leaf": np.linspace(0.1, 0.5, 100)
    }
    
    grid = RandomizedSearchCV(GradientBoostingRegressor(), param_distributions=param_grid,
                                n_iter=500, cv=3, verbose=3, n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid.best_estimator_

In [5]:
# X_train = df_A.drop(columns=["Limonene"]).values
# y_train = df_A["Limonene"].values

X_train = dfin.values
y_train = dfout.values.ravel()

In [6]:
# dtr = RandomForestRegressor()
# dtr.fit(X_train, y_train)

dtr = tune_and_train_rf(X_train, y_train)
# dtr = tune_and_train_gbt(X_train, y_train)

cv_score = cross_val_score(dtr, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
print(f"CV Score: {-cv_score.mean()}")

float_tensor_type = FloatTensorType([None, X_train.shape[1]])
initial_types = [("float_input", float_tensor_type)]
try:
    onnx_model = convert_sklearn(dtr, initial_types=initial_types)
except Exception:
    onnx_model = convert_xgboost(dtr, initial_types=initial_types)

results = pd.DataFrame({"pred": dtr.predict(X_train), "true": y_train})

Fitting 3 folds for each of 500 candidates, totalling 1500 fits
CV Score: 0.8072867370691753


In [46]:
def add_tree_model(opt_model, onnx_model, scaler, scaled_input_bounds):
    # init omlt block and gbt model based on the onnx format
    opt_model.gbt = OmltBlock()
    gbt_model = GradientBoostedTreeModel(onnx_model, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds)

    # omlt uses a big-m formulation to encode the tree models
    formulation = GBTBigMFormulation(gbt_model)
    opt_model.gbt.build_formulation(formulation)


opt_model = pe.ConcreteModel()
add_tree_model(opt_model, onnx_model, scaler, scaled_input_bounds)

# Add objective and constraints
opt_model.obj = pe.Objective(expr=opt_model.gbt.outputs[0], sense=pe.maximize)

Q40322_MENSP_idx = inputs.index('Q40322_MENSP')
opt_model.cons = pe.Constraint(expr=opt_model.gbt.inputs[Q40322_MENSP_idx] >= 8)

solver = pe.SolverFactory("ipopt")
status = solver.solve(opt_model, tee=True)

Ipopt 3.11.1: 
==> Warning: Treating 158 binary and 0 integer variables as continous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:      787
Number of nonzeros in inequality constraint Jacobian.:     2506
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      561
                     variables with only lower bounds:  

In [43]:
print(f"Objective value {opt_model.gbt.outputs[0].value}")

next_x = [opt_model.gbt.inputs[idx].value for idx in range(len(opt_model.gbt.inputs))]
next_x = pd.DataFrame(
    data=next_x, index=df_A.columns.drop("Limonene"), columns=["Value"]
)
next_x

Objective value 71.2060266994846


,Value
MVD1_YEAST,1.50620
Q40322_MENSP,9.00000
IDI_ECOLI,2.42820
ATOB_ECOLI,0.25095
Q8LKJ3_ABIGR,1.59150
Q9FD87_STAAU,0.27925
Q9FD86_STAAU,0.08545
KIME_YEAST,0.35120
ERG8_YEAST,0.48190
